In [1]:
# import some Python dependencies

try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
import json
import datetime
import csv
import time

In [2]:
access_token = 'EAACEdEose0cBAFqZAFNPj9k2ESFNavlcFXiUzy4m3epTRh5ykdPiGDOzmsvMjNSexsDFZCVOyJHMt9oV14P1oQZBcNda1s4ocKKzXokeyWYB68xa8Osq7CnONHpnXFynaAGlXQRJOUoI2hztIpuRjtvBZB4pZCVMk5nJCQhE75sqJBBmxIgZAh6o6HY46c4fxCFcZCXESE8owZDZD'

In [4]:
page_id = 'nba'

In [5]:
def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = urllib2.Request(url)
    response = urllib2.urlopen(req)
    data = json.loads(response.read())
    
    print (json.dumps(data, indent=4, sort_keys=True))
    

testFacebookPageData(page_id, access_token)

{
    "id": "8245623462",
    "name": "NBA"
}


In [6]:
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception as e:
            print (e)
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return response.read()

In [7]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    print (json.dumps(data, indent=4, sort_keys=True))
    

testFacebookPageFeedData(page_id, access_token)

{
    "data": [
        {
            "created_time": "2018-04-18T18:00:01+0000",
            "id": "8245623462_10156434716753463",
            "message": "Ahead of Houston Rockets x Minnesota Timberwolves Game 2 (9:30 pm/et, NBA on TNT) ... it's the BEST plays from James Harden's SPECTACULAR season! Who's winning tonight?? #NBAPlayoffs"
        },
        {
            "created_time": "2018-04-18T17:00:00+0000",
            "id": "8245623462_10156432020718463",
            "message": "\ud83d\udd8a\ufe0f Complete the sentence: after 36 PTS in Game 1, Paul George will have ______________ in Oklahoma City Thunder x Utah Jazz Game 2 TONIGHT. (8 pm/et, NBA TV)"
        },
        {
            "created_time": "2018-04-18T16:00:02+0000",
            "id": "8245623462_10156434677093463",
            "message": "\ud83d\udc51 x \ud83d\udeeb THE BEST buckets from LeBron James & Michael Jordan's #NBAPlayoffs careers, after King James moved past MJ into 2nd in postseason FG's in Game 1.\n\n\ud83c

In [8]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data
    

test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
print (json.dumps(test_status, indent=4, sort_keys=True))

{
    "comments": {
        "data": [
            {
                "created_time": "2018-04-18T18:12:16+0000",
                "id": "10156434716753463_10156435101538463",
                "message": "MVP\nMade in China \ud83d\ude02"
            }
        ],
        "paging": {
            "cursors": {
                "after": "OAZDZD",
                "before": "OAZDZD"
            },
            "next": "https://graph.facebook.com/v2.6/8245623462_10156434716753463/comments?access_token=EAACEdEose0cBAFqZAFNPj9k2ESFNavlcFXiUzy4m3epTRh5ykdPiGDOzmsvMjNSexsDFZCVOyJHMt9oV14P1oQZBcNda1s4ocKKzXokeyWYB68xa8Osq7CnONHpnXFynaAGlXQRJOUoI2hztIpuRjtvBZB4pZCVMk5nJCQhE75sqJBBmxIgZAh6o6HY46c4fxCFcZCXESE8owZDZD&summary=true&limit=1&after=OAZDZD"
        },
        "summary": {
            "can_comment": true,
            "order": "ranked",
            "total_count": 8
        }
    },
    "created_time": "2018-04-18T18:00:01+0000",
    "id": "8245623462_10156434716753463",
    "likes": {
        "data"

In [9]:
def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print (processed_test_status)

('8245623462_10156434716753463', b"Ahead of Houston Rockets x Minnesota Timberwolves Game 2 (9:30 pm/et, NBA on TNT) ... it's the BEST plays from James Harden's SPECTACULAR season! Who's winning tonight?? #NBAPlayoffs", b'BEST OF BEARD: James Harden Dishes, Dunks & Eurosteps To Amazing Year', 'video', 'https://www.facebook.com/nba/videos/10156434716753463/', '2018-04-18 13:00:01', 195, 8, 23)


In [10]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'w') as file:
        w = csv.writer(file)
        w.writerows(["status_id", "status_message", "link_name", "status_type", "status_link","status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print ("Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime))
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print ("%s Statuses Processed: %s" % (num_processed, datetime.datetime.now()))
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print ("\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime))


scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping nba Facebook Page: 2018-04-18 14:21:10.813284

1000 Statuses Processed: 2018-04-18 14:22:28.385085
HTTP Error 500: Internal Server Error
Error for URL https://graph.facebook.com/v2.6/8245623462/feed?fields=message,link,created_time,type,name,id,likes.limit%281%29.summary%28true%29,comments.limit%281%29.summary%28true%29,shares&limit=100&__paging_token=enc_AdDTWkNJsXLzbCDJT0wZBqXibkdZCZBFG1P4exGtURGZCNMDe23SY014MBf5ArzMWZA4WMx3XtVcm8nUGw5mQ7cfkp8bc&access_token=EAACEdEose0cBAFqZAFNPj9k2ESFNavlcFXiUzy4m3epTRh5ykdPiGDOzmsvMjNSexsDFZCVOyJHMt9oV14P1oQZBcNda1s4ocKKzXokeyWYB68xa8Osq7CnONHpnXFynaAGlXQRJOUoI2hztIpuRjtvBZB4pZCVMk5nJCQhE75sqJBBmxIgZAh6o6HY46c4fxCFcZCXESE8owZDZD&until=1434132001: 2018-04-18 14:24:04.885898
2000 Statuses Processed: 2018-04-18 14:24:16.038288
3000 Statuses Processed: 2018-04-18 14:26:25.750214
4000 Statuses Processed: 2018-04-18 14:27:27.648036
5000 Statuses Processed: 2018-04-18 14:27:47.319805

Done!
5638 Statuses Processed in 0:06:54.751149


In [11]:
import pandas as pd
import numpy as np

In [18]:
fb = pd.read_csv("nba_facebook.csv", sep = ',')
fb

,id,message,link_name,link_type,link,created_at,likes,comments,shares
0,8245623462_10156434716753463,"b""Ahead of Houston Rockets x Minnesota Timberw...","b'BEST OF BEARD: James Harden Dishes, Dunks & ...",video,https://www.facebook.com/nba/videos/1015643471...,2018/4/18 13:00,201.0,8.0,23.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8245623462_10156432020718463,b'\xf0\x9f\x96\x8a\xef\xb8\x8f Complete the se...,b'Paul George Plays Lights Out Game 1',video,https://www.facebook.com/nba/videos/1015643202...,2018/4/18 12:00,539.0,59.0,56.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8245623462_10156434677093463,"b""\xf0\x9f\x91\x91 x \xf0\x9f\x9b\xab THE BEST...","b""Michael Jordan & LeBron James' Greatest Play...",video,https://www.facebook.com/nba/videos/1015643467...,2018/4/18 11:00,2741.0,179.0,474.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8245623462_10156432046123463,"b""After LeBron James passed MJ in career posts...","b""Michael Jordan's BEST PLAY From Each Playoff...",video,https://www.facebook.com/nba/videos/1015643204...,2018/4/18 10:00,9118.0,353.0,2336.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8245623462_10156432163483463,"b""3\xef\xb8\x8f\xe2\x83\xa3 GAME 2\xef\xb8\x8f...","b""Photos from NBA's post""",photo,https://www.facebook.com/nba/photos/a.90133653...,2018/4/18 9:00,2575.0,97.0,240.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
raw_len_fb = len(fb)
raw_len_fb

11275

In [20]:
fb = fb.drop_duplicates()
fb = fb.dropna()
fb

,id,message,link_name,link_type,link,created_at,likes,comments,shares
0,8245623462_10156434716753463,"b""Ahead of Houston Rockets x Minnesota Timberw...","b'BEST OF BEARD: James Harden Dishes, Dunks & ...",video,https://www.facebook.com/nba/videos/1015643471...,2018/4/18 13:00,201.0,8.0,23.0
2,8245623462_10156432020718463,b'\xf0\x9f\x96\x8a\xef\xb8\x8f Complete the se...,b'Paul George Plays Lights Out Game 1',video,https://www.facebook.com/nba/videos/1015643202...,2018/4/18 12:00,539.0,59.0,56.0
4,8245623462_10156434677093463,"b""\xf0\x9f\x91\x91 x \xf0\x9f\x9b\xab THE BEST...","b""Michael Jordan & LeBron James' Greatest Play...",video,https://www.facebook.com/nba/videos/1015643467...,2018/4/18 11:00,2741.0,179.0,474.0
6,8245623462_10156432046123463,"b""After LeBron James passed MJ in career posts...","b""Michael Jordan's BEST PLAY From Each Playoff...",video,https://www.facebook.com/nba/videos/1015643204...,2018/4/18 10:00,9118.0,353.0,2336.0
8,8245623462_10156432163483463,"b""3\xef\xb8\x8f\xe2\x83\xa3 GAME 2\xef\xb8\x8f...","b""Photos from NBA's post""",photo,https://www.facebook.com/nba/photos/a.90133653...,2018/4/18 9:00,2575.0,97.0,240.0
10,8245623462_10156433570088463,b'The most dynamic plays from Jaylen Brown & T...,b'Best plays from Brown & Rozier in Games 1 & 2',video,https://www.facebook.com/nba/videos/1015643357...,2018/4/18 8:00,2940.0,66.0,185.0
12,8245623462_10156433430113463,"b""After leading the New Orleans Pelicans to a ...","b""The 6-10 Acrobatic: Anthony Davis' Most Athl...",video,https://www.facebook.com/nba/videos/1015643343...,2018/4/18 7:00,2223.0,45.0,142.0
14,8245623462_10156432128528463,"b'Stephen Curry, LeBron James, Kevin Durant, G...",b'Top Plays From Players With The Top 15 Most ...,video,https://www.facebook.com/nba/videos/1015643212...,2018/4/18 5:30,8194.0,105.0,907.0
16,8245623462_10156433459163463,"b""After leading the New Orleans Pelicans to a ...","b""Jrue Holiday's most athletic plays of the ye...",video,https://www.facebook.com/nba/videos/1015643345...,2018/4/18 4:30,2204.0,41.0,110.0
18,8245623462_10156433453993463,"b'Rajon Rondo (16 PTS, 10 REB, 9 AST) fell a d...",b'Rondo flirts with triple-double in Game 2',video,https://www.facebook.com/nba/videos/1015643345...,2018/4/18 3:30,6020.0,150.0,380.0


In [21]:
len(fb)

3822

In [22]:
fb.to_csv('nba_facebook_audit.csv', index=False)